In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

In [ ]:
vocab = 'абвгдежзийклмнопрстуфхцчшщъыьэюя'
vocab_to_int = {word: i + 1 for i, word in enumerate(vocab)}

# add padding
vocab_to_int['PAD'] = 0

# vocab size
vocab_size = len(vocab_to_int.keys())
vocab_size

def encode(text, max_char_length=20):
    encoded = np.zeros([max_char_length, vocab_size], dtype=int)
    i = 0
    for letter in text:
        if i >= max_char_length:
            break
        if letter.lower() in vocab_to_int:
            encoded[i][vocab_to_int[letter.lower()]] = 1
        i += 1
    return encoded

In [ ]:
def encode_word(word, max_char_length=20):
  word = encode(word)
  word = word.astype('float32')
  word = word.reshape(1, max_char_length, vocab_size)
  return word

наблюдник - словообразовательное,

рукотерник - словообразовательное,

улеч - грамматическое,

итти - фонетическое,

сумет - лексическое,

халупа - лексическое


In [ ]:
words = ['наблюдник',
        'рукотерник',
        'улеч',
        'итти',
        'мучонка',
        'сумет',
        'халупа']
test_data = {w: encode_word(w) for w in words}

In [ ]:
dec = ['Фонетическая', 'Словообразовательная', 'Лексическая', 'Грамматическая']

In [ ]:
model = load_model('cnn_uni.keras')
for word, encoded_word in test_data.items():
    prediction = model.predict(encoded_word)
    print(f'{word} принадлежит к категории: {np.argmax(prediction), dec[np.argmax(prediction)]}.')
    print(f'Вероятность по всем классам: {prediction}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
наблюдник принадлежит к категории: (1, 'Словообразовательное').
Вероятность по всем классам: [[0.04322779 0.78569305 0.1339468  0.03713239]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
рукотерник принадлежит к категории: (1, 'Словообразовательное').
Вероятность по всем классам: [[0.04981829 0.8059546  0.109111   0.03511619]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
улеч принадлежит к категории: (3, 'Грамматическое').
Вероятность по всем классам: [[0.25061318 0.16208398 0.28646523 0.30083764]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
итти принадлежит к категории: (2, 'Лексическое').
Вероятность по всем классам: [[0.25041297 0.13182154 0.3099734  0.30779216]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
мучонка принадлежит к категории: (1, 'Словообразовательное').
Вероятность по всем классам: [[0.07601828 0.4325802  0.35339296 0.13800855]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
сумет принадлежит к категории: (3, 'Грамматическое').
Вероятность по всем классам: [[0.25990